In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import tensorflow as tf
# import xgboost as xgb

## Lendo dados do DataSet

In [6]:
# Leitura dos Dataset
df = pd.read_csv("games.csv")
df.shape

(51490, 61)

In [7]:
#remoção das colunas desnecessárias
df = df.drop(['gameId', 'creationTime'], axis=1)
df

,gameDuration,seasonId,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,t1_champ1id,...,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills,t2_ban1,t2_ban2,t2_ban3,t2_ban4,t2_ban5
0,1949,9,1,2,1,1,1,1,2,8,...,5,0,0,1,1,114,67,43,16,51
1,1851,9,1,1,1,1,0,1,1,119,...,2,0,0,0,0,11,67,238,51,420
2,1493,9,1,2,1,1,1,2,0,18,...,2,0,0,1,0,157,238,121,57,28
3,1758,9,1,1,1,1,1,1,0,57,...,0,0,0,0,0,164,18,141,40,51
4,2094,9,1,2,1,1,1,1,0,19,...,3,0,0,1,0,86,11,201,122,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51485,1944,9,2,1,2,2,0,2,0,19,...,10,2,0,4,0,55,-1,90,238,157
51486,3304,9,2,1,1,2,2,2,2,90,...,11,7,4,4,1,157,55,119,154,105
51487,2156,9,2,2,2,2,0,1,0,141,...,10,2,0,2,0,113,122,53,11,157
51488,1475,9,2,2,2,2,0,2,0,121,...,11,3,0,1,0,154,39,51,90,114


In [8]:
df = df.fillna(0)
df

,gameDuration,seasonId,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,t1_champ1id,...,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills,t2_ban1,t2_ban2,t2_ban3,t2_ban4,t2_ban5
0,1949,9,1,2,1,1,1,1,2,8,...,5,0,0,1,1,114,67,43,16,51
1,1851,9,1,1,1,1,0,1,1,119,...,2,0,0,0,0,11,67,238,51,420
2,1493,9,1,2,1,1,1,2,0,18,...,2,0,0,1,0,157,238,121,57,28
3,1758,9,1,1,1,1,1,1,0,57,...,0,0,0,0,0,164,18,141,40,51
4,2094,9,1,2,1,1,1,1,0,19,...,3,0,0,1,0,86,11,201,122,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51485,1944,9,2,1,2,2,0,2,0,19,...,10,2,0,4,0,55,-1,90,238,157
51486,3304,9,2,1,1,2,2,2,2,90,...,11,7,4,4,1,157,55,119,154,105
51487,2156,9,2,2,2,2,0,1,0,141,...,10,2,0,2,0,113,122,53,11,157
51488,1475,9,2,2,2,2,0,2,0,121,...,11,3,0,1,0,154,39,51,90,114


## Lendo os Dados do Arquivos auxiliares

* summoner_spell_info.json
* champion_info_2.json

In [9]:
champions_info_df = pd.read_json('champions_info_modified.json')


# print(champions_info_df.head())
# print(summoner_spell_info_df.head())
champions_info_df = champions_info_df['data'] 
champions_info_transposed = champions_info_df.transpose()
print(champions_info_transposed)
# Lendo o arquivo summoner_spell_info.json
summoner_spell_info_df = pd.read_json('summoner_spell_info.json')

-1      {'tags': [], 'title': 'None', 'id': -1, 'key':...
 1      {'tags': ['Mage'], 'title': 'the Dark Child', ...
 10     {'tags': ['Fighter', 'Support'], 'title': 'The...
 101    {'tags': ['Mage', 'Assassin'], 'title': 'the M...
 102    {'tags': ['Fighter', 'Tank'], 'title': 'the Ha...
                              ...                        
 91     {'tags': ['Assassin', 'Fighter'], 'title': 'th...
 92     {'tags': ['Fighter', 'Assassin'], 'title': 'th...
 96     {'tags': ['Marksman', 'Mage'], 'title': 'the M...
 98     {'tags': ['Tank'], 'title': 'the Eye of Twilig...
 99     {'tags': ['Mage', 'Support'], 'title': 'the La...
Name: data, Length: 139, dtype: object


In [10]:
#Pegando a coluna a ser analisada(Quem venceu)
x = df.drop(['winner'], axis=1).values 
y = df['winner']

In [11]:
# Separando Teste(20) e Treino(80)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [214]:
# Escalonando


# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(trainData)
# X_test_scaled = scaler.transform(testData)

In [16]:

# Modelos
# Decision Tree
# KNN
# Regrassão Logistica
# Random Forest
# Redes Neurais
# 
# Criando o objeto do modelo KNN



def KNNTrain(X_train,X_test, y_train,y_test , k):
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(X_train,y_train)
    y_pred  = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("k = {}, accuracy = {:.2f}%".format(k,accuracy*100))
    return accuracy


In [17]:
KNNTrain(X_train,X_test, y_train,y_test , 3)

k = 3, accuracy = 52.15%


0.5214604777626723

### Árvore de Decisão

In [20]:
def DecisionTreeTrain(X_train,X_test, y_train,y_test):
    # Inicializa o classificador de árvore de decisão
    model = DecisionTreeClassifier()
    # Treina o modelo com os dados de treinamento
    model.fit(X_train, y_train)
    # Realiza a predição nos dados de teste
    y_pred = model.predict(X_test)
    # Calcula a acurácia do modelo
    accuracy = accuracy_score(y_test, y_pred)
    print("accuracy = {:.2f}%".format(accuracy*100))
    return accuracy

In [21]:
DecisionTreeTrain(X_train,X_test, y_train,y_test)

accuracy = 95.85%


0.9585356379879588

### Random Florest

In [22]:
def RandomForestTrain(X_train,X_test, y_train,y_test):
    # Inicializa o classificador de árvore de decisão
    model = RandomForestClassifier()
    # Treina o modelo com os dados de treinamento
    model.fit(X_train, y_train)
    # Realiza a predição nos dados de teste
    y_pred = model.predict(X_test)
    # Calcula a acurácia do modelo
    accuracy = accuracy_score(y_test, y_pred)
    print("accuracy = {:.2f}%".format(accuracy*100))
    return accuracy

In [23]:
RandomForestTrain(X_train,X_test, y_train,y_test)

accuracy = 96.91%


0.9691202175179646

### Regreção Logistica

In [18]:
def LogisticRegressionTrain(X_train,X_test, y_train,y_test):
    # Criar um objeto de regressão logística
    model = LogisticRegression()

    # Treinar o modelo de regressão logística usando os dados de treinamento
    model.fit(X_train, y_train)

    # Fazer previsões usando os dados de teste
    y_pred = model.predict(X_test)

    # Calcular a acurácia do modelo
    accuracy = accuracy_score(y_test, y_pred)
    print("accuracy = {:.2f}%".format(accuracy*100))
    return accuracy

In [19]:
LogisticRegressionTrain(X_train,X_test, y_train,y_test)

accuracy = 95.81%


c:\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9580501068168576

In [ ]:
# Definir os parâmetros do modelo
params = {
    'objective': 'binary:logistic',  # Para classificação binária
    'eval_metric': 'logloss',
    'eta': 0.1,
    'max_depth': 3,
    'n_estimators': 100
}
# Criar o objeto do modelo de Gradient Boosting
model = xgb.XGBClassifier(**params)

# Treinar o modelo
model.fit(X_train, y_train)

# Fazer previsões usando os dados de teste
y_pred = model.predict(X_test)

# Calcular a acurácia do modelo
accuracy = accuracy_score(y_test, y_pred)
print("Acurácia do modelo: {:.2f}".format(accuracy))

### Neural Network

In [24]:
def NeuralNetworkTrain(X_train, X_test, y_train, y_test, input_dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    # Compilar o modelo
    model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])
    # Treinar o modelo
    model.fit(X_train, y_train, epochs=10, batch_size=32)
    # Fazer previsões usando os dados de teste
    y_pred = model.predict(X_test)
    y_pred = (y_pred > 0.5)  # Converter as previsões em rótulos binários
    # Calcular a acurácia do modelo
    accuracy = accuracy_score(y_test, y_pred)
    print("Acurácia do modelo: {:.2f}".format(accuracy))
    return accuracy

In [25]:
feature_names = df.drop(['winner'], axis=1).columns.tolist()
input_dim = len(feature_names)
NeuralNetworkTrain(X_train, X_test, y_train, y_test, input_dim)

Epoch 1/10
1288/1288 [==============================] - 2s 1ms/step - loss: -17340620.0000 - accuracy: 0.5060
Epoch 2/10
1288/1288 [==============================] - 2s 2ms/step - loss: -226012192.0000 - accuracy: 0.5061
Epoch 3/10
1288/1288 [==============================] - 2s 1ms/step - loss: -816876288.0000 - accuracy: 0.5061
Epoch 4/10
1288/1288 [==============================] - 2s 2ms/step - loss: -1870180864.0000 - accuracy: 0.5061
Epoch 5/10
1288/1288 [==============================] - 2s 2ms/step - loss: -3449982720.0000 - accuracy: 0.5061
Epoch 6/10
1288/1288 [==============================] - 2s 2ms/step - loss: -5620452352.0000 - accuracy: 0.5061
Epoch 7/10
1288/1288 [==============================] - 2s 1ms/step - loss: -8447165952.0000 - accuracy: 0.5061
Epoch 8/10
1288/1288 [==============================] - 2s 2ms/step - loss: -11992857600.0000 - accuracy: 0.5061
Epoch 9/10
1288/1288 [==============================] - 2s 1ms/step - loss: -16317168640.0000 - accuracy: 0